In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')

In [56]:
base_path = './preprocessed data/'

train_err = pd.read_csv(base_path + 'new_train_err.csv', parse_dates=['time'])
train_quality = pd.read_csv(base_path + 'new_train_quality.csv', parse_dates=['time'])
train_problem = pd.read_csv(base_path + 'new_train_problem.csv', parse_dates=['time'])
test_err = pd.read_csv(base_path + 'new_test_err.csv', parse_dates=['time'])
test_quality = pd.read_csv(base_path + 'new_test_quality.csv', parse_dates=['time'])
submission = pd.read_csv("C:/Users/gmlkd/data/시스템 품질 변화로 인한 사용자 불편 예지 AI 경진대회_data/sample_submission.csv")

In [57]:
change_model =  train_err[['user_id', 'model_nm']]

In [58]:
change_model = change_model.drop_duplicates(subset=None, keep='first')

In [59]:
user_model_num = change_model.user_id.value_counts(sort=False)
user_has_2models = user_model_num > 1

In [60]:
train_err.drop(columns=['time', 'errcode'], inplace=True)
train_err

,user_id,model_nm,fwver,errtype
0,10000,model_3,05.15.2138,15
1,10000,model_3,05.15.2138,12
2,10000,model_3,05.15.2138,11
3,10000,model_3,05.15.2138,16
4,10000,model_3,05.15.2138,4
...,...,...,...,...
15367996,24999,model_3,05.15.2138,15
15367997,24999,model_3,05.15.2138,16
15367998,24999,model_3,05.15.2138,4
15367999,24999,model_3,05.15.2138,4


In [61]:
train_err = pd.get_dummies(data=train_err, columns=['model_nm', 'fwver', 'errtype'])
train_err.head()

,user_id,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,...,errtype_33,errtype_34,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42
0,10000,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10000,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10000,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10000,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10000,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
train_err = train_err.groupby(['user_id']).sum()
print(train_err.shape)
train_err.head()

(15000, 87)


,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,fwver_03.11.1141,...,errtype_33,errtype_34,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42
user_id,,,,,,,,,,,,,,,,,,,,,
10000,0.0,0.0,0.0,316.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10001,0.0,0.0,2345.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,18.0,0.0,1.0,1.0,0.0,0.0,113.0,56.0,1.0
10002,0.0,0.0,0.0,305.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10003,0.0,0.0,297.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.0,0.0,0.0,1.0,1.0,2.0,0.0,17.0,1.0,0.0
10004,777.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,16.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0,0.0,2.0


In [66]:
user_has_2models = user_has_2models.astype(int)
user_has_2models = user_has_2models.to_frame(name='multiple_models')

In [67]:
train_err = pd.concat([train_err, user_has_2models], axis=1)                     

,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,fwver_03.11.1141,...,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,user_id,multiple_models
10000,0.0,0.0,0.0,316.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
10001,0.0,0.0,2345.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,113.0,56.0,1.0,0,0
10002,0.0,0.0,0.0,305.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
10003,0.0,0.0,297.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,2.0,0.0,17.0,1.0,0.0,0,0
10004,777.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,4.0,0.0,2.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0.0,0.0,191.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,9.0,7.0,4.0,0,0
24996,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
24997,818.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,58.0,7.0,5.0,0,0
24998,144.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,6.0,0.0,0.0,0,0


In [71]:
train_err

,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,fwver_03.11.1141,...,errtype_34,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,multiple_models
10000,0.0,0.0,0.0,316.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
10001,0.0,0.0,2345.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,18.0,0.0,1.0,1.0,0.0,0.0,113.0,56.0,1.0,0
10002,0.0,0.0,0.0,305.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
10003,0.0,0.0,297.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,2.0,0.0,17.0,1.0,0.0,0
10004,777.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,4.0,0.0,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0.0,0.0,191.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9.0,7.0,4.0,0
24996,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
24997,818.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,12.0,0.0,1.0,1.0,0.0,0.0,58.0,7.0,5.0,0
24998,144.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,1.0,1.0,0.0,0.0,6.0,0.0,0.0,0


In [72]:
problem = np.zeros(15000)
problem[train_problem.user_id.unique() - 10000] = 1
problem

array([0., 1., 0., ..., 1., 1., 0.])

In [80]:
X = train_err
y = problem

print(X.shape, y.shape)

(15000, 88) (15000,)


In [79]:
from xgboost import XGBClassifier              # 1. XGBoost
from lightgbm.sklearn import LGBMClassifier    # 2. LightGBM

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [81]:
model1 = XGBClassifier()
model1.fit(X,y)
pred1 = model1.predict(X)
print(  confusion_matrix(y,pred1),  "\n")
print(  classification_report(y,pred1)  )

[[9869  131]
 [1466 3534]] 

              precision    recall  f1-score   support

         0.0       0.87      0.99      0.93     10000
         1.0       0.96      0.71      0.82      5000

    accuracy                           0.89     15000
   macro avg       0.92      0.85      0.87     15000
weighted avg       0.90      0.89      0.89     15000



In [82]:
model2 = LGBMClassifier()
model2.fit(X,y)
pred2 = model2.predict(X)
print(  confusion_matrix(y,pred2),  "\n")
print(  classification_report(y,pred2)  )

[[9701  299]
 [1956 3044]] 

              precision    recall  f1-score   support

         0.0       0.83      0.97      0.90     10000
         1.0       0.91      0.61      0.73      5000

    accuracy                           0.85     15000
   macro avg       0.87      0.79      0.81     15000
weighted avg       0.86      0.85      0.84     15000

